In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt

from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
float_data = pd.read_csv('../input/jena-climate-2009-2016/jena_climate_2009_2016.csv')

In [ ]:
float_data = float_data.iloc[:4000,1:].to_numpy()

In [ ]:
temp = float_data[:, 1]
plt.plot(range(len(temp)), temp)

In [ ]:
plt.plot(range(1440), temp[:1440])

In [ ]:
mean = float_data[:2000].mean(axis=0)
float_data -= mean
std = float_data[:2000].std(axis=0)
float_data /= std

## Generator yielding temporal samples

In [ ]:
def generator(data, lookback, delay, min_index, max_index,
        shuffle=False, batch_size=128, step=6):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(
                min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)
        samples = np.zeros((len(rows),
            lookback // step,
            data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]
        yield samples, targets

In [ ]:
lookback = 144
step = 6
delay = 144
batch_size = 128

train_gen = generator(float_data,
    lookback=lookback,
    delay=delay,
    min_index=0,
    max_index=2000,
    shuffle=True,
    step=step,
    batch_size=batch_size)
val_gen = generator(float_data,
    lookback=lookback,
    delay=delay,
    min_index=2001,
    max_index=3000,
    step=step,
    batch_size=batch_size)
test_gen = generator(float_data,
    lookback=lookback,
    delay=delay,
    min_index=3001,
    max_index=None,
    step=step,
    batch_size=batch_size)
val_steps = (3000 - 2001 - lookback)
test_steps = (len(float_data) - 3001 - lookback)

## Baseline model

In [ ]:
def evaluate_naive_method():
    batch_maes = []
    for step in range(val_steps):
        samples, targets = next(val_gen)
        preds = samples[:, -1, 1]
        mae = np.mean(np.abs(preds - targets))
        batch_maes.append(mae)
    print(np.mean(batch_maes))
evaluate_naive_method()

In [ ]:
celsius_mae = 0.29 * std[1]

## Basic ML model

In [ ]:
model = Sequential()
model.add(layers.Flatten(input_shape=(lookback // step, float_data.shape[-1])))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1))
model.compile(optimizer=RMSprop(), loss='mae')
history = model.fit_generator(train_gen,
    steps_per_epoch=50,
    epochs=10,
    validation_data=val_gen,
    validation_steps=val_steps)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

## GRU model

In [ ]:
model = Sequential()
model.add(layers.GRU(32, input_shape=(None, float_data.shape[-1])))
model.add(layers.Dense(1))
model.compile(optimizer=RMSprop(), loss='mae')
history = model.fit_generator(train_gen,
    steps_per_epoch=50,
    epochs=10,
    validation_data=val_gen,
    validation_steps=val_steps)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

## GRU with dropout

In [ ]:
model = Sequential()
model.add(layers.GRU(32,
    dropout=0.2,
    recurrent_dropout=0.2,
    input_shape=(None, float_data.shape[-1])))
model.add(layers.Dense(1))
model.compile(optimizer=RMSprop(), loss='mae')
history = model.fit_generator(train_gen,
    steps_per_epoch=50,
    epochs=10,
    validation_data=val_gen,
    validation_steps=val_steps)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
model = Sequential()
model.add(layers.GRU(32,
    dropout=0.1,
    recurrent_dropout=0.5,
    return_sequences=True,
    input_shape=(None, float_data.shape[-1])))
model.add(layers.GRU(64, activation='relu',
    dropout=0.1,
    recurrent_dropout=0.5))
model.add(layers.Dense(1))
model.compile(optimizer=RMSprop(), loss='mae')
history = model.fit_generator(train_gen,
    steps_per_epoch=50,
    epochs=10,
    validation_data=val_gen,
    validation_steps=val_steps)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

## Bidirectional GRU

In [ ]:
model = Sequential()
model.add(layers.Bidirectional(
    layers.GRU(32), input_shape=(None, float_data.shape[-1])))
model.add(layers.Dense(1))
model.compile(optimizer=RMSprop(), loss='mae')
history = model.fit_generator(train_gen,
    steps_per_epoch=50,
    epochs=10,
    validation_data=val_gen,
    validation_steps=val_steps)

## 1D Convnet

In [ ]:
model = Sequential()
model.add(layers.Conv1D(32, 5, activation='relu',
    input_shape=(None, float_data.shape[-1])))
model.add(layers.MaxPooling1D(3))
#model.add(layers.Conv1D(32, 5, activation='relu'))
#model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.compile(optimizer=RMSprop(), loss='mae')
history = model.fit_generator(train_gen,
    steps_per_epoch=50,
    epochs=10,
    validation_data=val_gen,
    validation_steps=val_steps)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

## Higher-resolution generator

In [ ]:
step = 3
lookback = 72
delay = 144
train_gen = generator(float_data,
    lookback=lookback,
    delay=delay,
    min_index=0,
    max_index=2000,
    shuffle=True,
    step=step)
val_gen = generator(float_data,
    lookback=lookback,
    delay=delay,
    min_index=2001,
    max_index=3000,
    step=step)
test_gen = generator(float_data,
    lookback=lookback,
    delay=delay,
    min_index=3001,
    max_index=None,
    step=step)
val_steps = (3000 - 2001 - lookback) // 128
test_steps = (len(float_data) - 3001 - lookback) // 128

## 1D convolution + GRU

In [ ]:
model = Sequential()
model.add(layers.Conv1D(32, 5, activation='relu',
    input_shape=(None, float_data.shape[-1])))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.GRU(32, dropout=0.1, recurrent_dropout=0.5))
model.add(layers.Dense(1))
model.summary()
model.compile(optimizer=RMSprop(), loss='mae')

history = model.fit_generator(train_gen,
    steps_per_epoch=50,
    epochs=10,
    validation_data=val_gen,
    validation_steps=val_steps)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()